# Hypotesis and Inference

## Statistical Hypothesis Testing

Imagine we have a coin and we want to test whether it’s fair. We’ll make the assumption that the coin has some probability $p$ of landing heads, and so our null hypothesis is that the coin is fair $p = 0 . 5$. We’ll test this against the alternative 
hypothesis p ≠ 0 . 5

HP: $p = 0 . 5$

In particular, our test will involve flipping the coin some number n times and count
ing the number of heads X. Each coin flip is a Bernoulli trial, which means that X is  
Binomial(n,p) random variable, whi6) we can approxima e
using the normal distribution:.

In [2]:
import math 

def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma


In particular, let’s say that we choose to flip the coin n = 1000 times. If our hypothesis of fairness is true, X should be distributed approximately normally with mean 50 and
standard deviation 15.8:

In [4]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
print(mu_0, sigma_0)

500.0 15.811388300841896


We need to make a decision about __significance__: how willing we are to make a type 1
error (“false positive”), in which we reject H0
even though it’s true. For reasons lost to
the annals of history, this willingness is often set at 5% or 1%. Let’s choose 5%

In [22]:
def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2 
    
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """find approximate inverse using binary search"""
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    else:
        low_z, low_p = -10.0, 0 # normal_cdf(-10) is (very close to) 0
        hi_z, hi_p = 10.0, 1 # normal_cdf(10) is (very close to) 1
        while hi_z - low_z > tolerance:
            mid_z = (low_z + hi_z) / 2 # consider the midpoint
            mid_p = normal_cdf(mid_z) # and the cdf's value there
            if mid_p < p:
                # midpoint is still too low, search above it
                low_z, low_p = mid_z, mid_p
            elif mid_p > p:
                # midpoint is still too high, search below it
                hi_z, hi_p = mid_z, mid_p
            else:
                break
    return mid_z

def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)
def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)
    
def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds
    that contain the specified probability"""
    tail_probability = (1 - probability) / 2
     # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    return lower_bound, upper_bound

lower_bound, upper_bound=normal_two_sided_bounds(0.95, mu_0, sigma_0) 
print('lower_bound, upper_bound=', lower_bound, upper_bound)

lower_bound, upper_bound= 469.01026640487555 530.9897335951244


We are also often interested in the __power__ of a test, which is the probability of not
making a type 2 error, in which we fail to reject H0
 even though it’s false. In order to
measure this, we have to specify what exactly H0
being false means. (Knowing merely
that p is not 0.5 doesn’t give you a ton of information about the distribution of X.) In
particular, let’s check what happens if p is really 0.55, so that the coin is slightly biased
toward heads.

In [24]:
# actual mu and sigma based on p = 0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
# a type 2 error means we fail to reject the null hypothesis
# which will happen when X is still in our original interval
type_2_probability = normal_cdf(upper_bound, mu_1, sigma_1) - normal_cdf(lower_bound, mu_1, sigma_1)
power = 1 - type_2_probability 
print('power=', power)

power= 0.8865480012953671


## Confidence Intervals

we can estimate the probability of the unfair coin by looking at the aver‐
age value of the Bernoulli variables corresponding to each flip—1 if heads, 0 if tails. If
we observe 525 heads out of 1,000 flips, then we estimate $p = 0.525$

How confident can we be about this estimate? Well, if we knew the exact value of p,
the central limit theore) tells us
that the average of those Bernoulli variables should be approximately normal, with
mean p and standard deviation:

In [26]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print('mu=', mu, 'sigma=', sigma)

mu= 0.525 sigma= 0.015791611697353755


we are “95% confident” that the following interval
contains the true parameter p:


In [28]:
lower_bound, upper_bound=normal_two_sided_bounds(0.95, mu, sigma)
print('lower_bound, upper_bound=', lower_bound, upper_bound)


lower_bound, upper_bound= 0.4940490278129096 0.5559509721870904


0.5 falls within our confidence interval